In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def scrape_parliament():
    driver = webdriver.Firefox()  # or Chrome()
    driver.get("https://www.hellenicparliament.gr/Vouleftes/Viografika-Stoicheia/")
    
    all_data = []
    page = 1
    
    while True:
        # Wait for table to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "tr")))
        
        # Extract data from current page
        rows = driver.find_elements(By.CSS_SELECTOR, "tr")
        for row in rows:
            try:
                cols = row.find_elements(By.CSS_SELECTOR, "td")
                if len(cols) >= 4:
                    party = cols[0].find_element(By.CSS_SELECTOR, "img").get_attribute("alt")
                    name = cols[1].text
                    region = cols[2].text
                    contact = cols[3].text
                    
                    all_data.append({
                        'party': party,
                        'name': name,
                        'region': region,
                        'contact': contact
                    })
            except:
                continue
                
        # Check if there's a next page
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a[title='Επόμενη σελίδα']")
            next_button.click()
            page += 1
            time.sleep(2)  # Wait for page to load
        except:
            break
    
    driver.quit()
    return pd.DataFrame(all_data)

# Run scraper
df = scrape_parliament()
df.to_csv('parliament_members.csv', index=False)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def scrape_parliament():
    driver = webdriver.Firefox()
    driver.get("https://www.hellenicparliament.gr/Vouleftes/Viografika-Stoicheia/")
    all_data = []
    
    for page in range(1, 31):  # 30 pages total
        driver.get(f"https://www.hellenicparliament.gr/Vouleftes/Viografika-Stoicheia/?SortBy=name&SortDirection=asc&pageNo={page}")
        time.sleep(2)
        
        rows = driver.find_elements(By.CSS_SELECTOR, "tr")
        for row in rows:
            try:
                cols = row.find_elements(By.CSS_SELECTOR, "td")
                if len(cols) >= 4:
                    party = cols[0].find_element(By.CSS_SELECTOR, "img").get_attribute("alt")
                    name = cols[1].text
                    region = cols[2].text
                    contact = cols[3].text
                    all_data.append({'party': party, 'name': name, 'region': region, 'contact': contact})
            except:
                continue

    driver.quit()
    return pd.DataFrame(all_data)

df = scrape_parliament()
df.to_csv('parliament_members.csv', index=False)